In [ ]:
from typing import TypedDict, List, Annotated
import operator
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState, START, END
from IPython.display import Image, display
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("OPENAI_ENDPOINT")
model_name = "gpt-4o-mini"
temp=0.0

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model=model_name,
    temperature=temp
)


In [ ]:
class State(TypedDict):
    value: Annotated[List[str], operator.add]

def node_a(state: State):
    return {"value": ["a"]}

def node_b(state: State):
    return {"value": ["b"]}

workflow = StateGraph(State)

workflow.add_node(node_a)
workflow.add_node(node_b)

workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

graph = workflow.compile()

display(
    Image(
        graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
graph.invoke({"value":[]})

**Checkpoints**

In [ ]:
checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

In [ ]:
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"value": []}, config)

In [ ]:
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"value": []}, config)

In [ ]:
config = {"configurable": {"thread_id": "2"}}
graph.invoke({"value": []}, config)

In [ ]:
config = {"configurable": {"thread_id": "3"}}
graph.invoke({"value": []}, config)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": "1"}}))

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": "2"}}))